In [7]:
import pandas as pd
import sys
sys.path.append('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/src')
from algorithms.battery_optimize import battery_optimisation

def optimize_battery_operation_every_week(
    da_price_data,
    max_battery_capacity,
    initial_capacity,
    end_capacity,
    max_battery_power,
    efficiency,
    daily_max_charging_circles,
    include_revenue,
    solver='glpk'
):
    """
    Optimise the battery operation for every week in the given data
    """
    # Convert datetime to datetime64[ns]
    da_price_data['datetime'] = da_price_data['datetime'].astype('datetime64[ns]')
    
    # Group by week
    da_price_data['year'] = da_price_data['datetime'].dt.year
    da_price_data['week'] = da_price_data['datetime'].dt.isocalendar().week

    # Optimise for each week for every year
    for year in da_price_data['year'].unique():
        year_data = da_price_data[da_price_data['year'] == year]
        total_yearly_result_df = pd.DataFrame()
        for week in year_data['week'].unique():
            week_data = year_data[year_data['week'] == week]
            result = battery_optimisation(
                week_data['datetime'], 
                week_data['da_price'], 
                max_battery_capacity=max_battery_capacity,
                initial_capacity=initial_capacity,
                end_capacity=end_capacity,
                max_battery_power=max_battery_power,
                efficiency=efficiency,
                daily_max_charging_circles=daily_max_charging_circles,
                include_revenue=include_revenue,    
                solver=solver
            )
            result.to_csv(f'/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/da_optimization/weekly_optimization/battery_operation_{year}_{week}.csv', index=False)
            revenue = result['revenue'].sum()
            print(f'Year: {year}, Week: {week}, Revenue: {revenue}')
            total_yearly_result_df = pd.concat([total_yearly_result_df, result])
        total_yearly_result_df.to_csv(f'/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/da_optimization/yearly_optimization/battery_operation_{year}.csv', index=False)

In [8]:
da_price_2024 = pd.read_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/processed/day_ahead/da_price_2024.csv')
da_price_2024.datetime = pd.to_datetime(da_price_2024.datetime)

In [9]:
da_price_2024.head()

,datetime,da_price
0,2024-01-01 00:00:00,39.91
1,2024-01-01 00:15:00,-0.04
2,2024-01-01 00:30:00,-9.01
3,2024-01-01 00:45:00,-29.91
4,2024-01-01 01:00:00,25.28


In [10]:
optimize_battery_operation_every_week(
    da_price_2024,
    max_battery_capacity=2,
    initial_capacity=1, 
    end_capacity=1,
    max_battery_power=1,
    efficiency=0.95,
    daily_max_charging_circles=20000000,
    include_revenue=True,
    solver='glpk')

Year: 2024, Week: 1, Revenue: 2586.5025720394733
Year: 2024, Week: 2, Revenue: 1824.6714891447368
Year: 2024, Week: 3, Revenue: 2093.105540131579
Year: 2024, Week: 4, Revenue: 2055.110385526316
Year: 2024, Week: 5, Revenue: 1597.324675986842
Year: 2024, Week: 6, Revenue: 1656.6140934210525
Year: 2024, Week: 7, Revenue: 1552.815239144737
Year: 2024, Week: 8, Revenue: 1864.6515167763157
Year: 2024, Week: 9, Revenue: 1692.248907565789
Year: 2024, Week: 10, Revenue: 1710.9867309210524
Year: 2024, Week: 11, Revenue: 1830.9844881578947
Year: 2024, Week: 12, Revenue: 1967.6035398026315
Year: 2024, Week: 13, Revenue: 2737.5208628289474
Year: 2024, Week: 14, Revenue: 2561.3714375
Year: 2024, Week: 15, Revenue: 3498.228976315789
Year: 2024, Week: 16, Revenue: 2838.289664473684
Year: 2024, Week: 17, Revenue: 2868.583496710526
Year: 2024, Week: 18, Revenue: 3571.438254605263
Year: 2024, Week: 19, Revenue: 3138.5919427631575
Year: 2024, Week: 20, Revenue: 3587.663293421053
Year: 2024, Week: 21, Rev

In [12]:
df_2024_revenue = pd.read_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/da_optimization/yearly_optimization/battery_operation_2024.csv')
df_2024_revenue.head()
df_2024_revenue.revenue.sum()

np.float64(150381.13849210527)

In [13]:
df_2024_first_half = df_2024_revenue[df_2024_revenue.datetime < '2024-07-01']

In [14]:
df_2024_first_half.revenue.sum()

np.float64(66897.63078421052)

In [19]:
df_2024_revenue.head()

,datetime,spot_price,power,market_dispatch,opening_capacity,revenue
0,2024-01-01 00:00:00,39.91,1.0,0.2375,1.0000,9.478625
1,2024-01-01 00:15:00,-0.04,0.0,0.0000,0.7500,-0.000000
2,2024-01-01 00:30:00,-9.01,0.0,0.0000,0.7500,-0.000000
3,2024-01-01 00:45:00,-29.91,-1.0,-0.2500,0.7500,7.477500
4,2024-01-01 01:00:00,25.28,1.0,0.2375,0.9875,6.004000


In [15]:
circles = df_2024_revenue[df_2024_revenue['market_dispatch'] > 0].market_dispatch.sum()/2
print('total circles: ', circles)

total circles:  1430.4684374999997
